In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
df_gscore = pd.read_csv('genome-scores.csv')
df_gtags = pd.read_csv('genome-tags.csv')
df_links = pd.read_csv('links.csv')
df_movies = pd.read_csv('movies.csv')
df_rating = pd.read_csv('ratings.csv')
df_tagspd = pd.read_csv('tags.csv')

In [9]:
df_gscore.movieId.nunique()

10381

## Creating the genome dataset

In [256]:
df_gtagscore=df_gtags.merge(df_gscore, how='right', on='tagId').drop('tag', axis =1)
df_gtagscore.drop_duplicates(subset=['tagId', 'movieId'],inplace=True)
df_gtagscore.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Non-Null Count     Dtype  
---  ------     --------------     -----  
 0   tagId      11709768 non-null  int64  
 1   movieId    11709768 non-null  int64  
 2   relevance  11709768 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 357.4 MB


# pivoting the dataframe

In [257]:
df_gtagscore = df_gtagscore.pivot(index = 'movieId', columns = 'tagId', values='relevance')

In [258]:
df_gtagscore.shape

(10381, 1128)

## adding realease year and genres as columns

In [259]:
df_movies = pd.read_csv('movies.csv')
mask = df_movies['title'].str.contains('09–')

df_movies = pd.read_csv('movies.csv')
def condition(x):
    if x[-2:]=='a)':
        return np.nan
    elif x[-2:]=='l)':
        return np.nan
    elif x[-3:-1]=='7-':
        return 2007
    elif x[-4:-2]=='9–':
        return 2009
    elif x[-2:]=='))':
        return x[-6:-2]
    elif x[-1:]==')':
        return x[-5:-1]
    elif x[-1:]==' ':
        return x[-6:-2]
    else:
        return np.nan
df_movies['releaseyear'] = df_movies['title'].apply(condition).fillna(1993)

In [260]:
df_movies.genres= df_movies.genres.str.split('|')
dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).groupby(level=0).sum()
df_movies = pd.concat([df_movies, dummies], axis=1).drop('genres', axis=1)

In [261]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
df_movies =df_movies.drop('title', axis =1)
"""
le = LabelEncoder()
le.fit(df_movies.genres)
df_movies.genres= le.transform(df_movies.genres)

scaler = MinMaxScaler()
df_movies.genres = scaler.fit_transform(df_movies[['genres']])
"""
scaler02 = MinMaxScaler()
df_movies.releaseyear = scaler02.fit_transform(df_movies[['releaseyear']])

In [272]:
df_ContBaseRec = pd.merge(df_gtagscore, df_movies, how='inner', on='movieId').set_index('movieId')
df_ContBaseRec.columns = df_ContBaseRec.columns.astype(str)

In [268]:
df_ContBaseRec.shape

(27278, 1149)

In [277]:
df_ContBaseRec

,1,2,3,4,5,6,7,8,9,10,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,0.03200,...,0,0,0,0,0,0,0,0,0,0
2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,0.02050,...,0,0,0,0,0,0,0,0,0,0
3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,0.02675,...,0,0,0,0,0,1,0,0,0,0
4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,0.03025,...,0,0,0,0,0,1,0,0,0,0
5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,0.02875,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130578,0.15400,0.04025,0.01975,0.01850,0.02850,0.02275,0.01000,0.03675,0.01025,0.01575,...,0,0,0,0,0,0,0,1,0,0
130840,0.02400,0.03025,0.02425,0.09375,0.05600,0.06025,0.02100,0.12900,0.11275,0.03650,...,0,1,0,0,0,1,1,0,0,0
131013,0.04200,0.03175,0.01700,0.05600,0.05750,0.02675,0.01600,0.04200,0.02800,0.03725,...,0,0,0,0,0,0,0,0,0,0


# Clustering

## importing the libraries

In [274]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

## training the Model

In [275]:
kmeans = KMeans(n_clusters=18, n_init='auto')
kmeans.fit(df_ContBaseRec)
labels = kmeans.labels_

In [280]:
df_movies = pd.read_csv('movies.csv')
df_labeledMovies= pd.DataFrame({'movieId': df_ContBaseRec.index, 'labels': labels}).merge(df_movies, on='movieId').set_index('movieId')

# recommending the nearest neighbors within the given cluster with the example "Network (1975)

In [12]:
from sklearn.neighbors import NearestNeighbors

In [16]:
#specifying the movie
given_movie_id =df_labeledMovies[df_labeledMovies.title.str.contains(r'\bNetwork\b.{0,9}$', case=False)].labels.item()
given_movie_cluster = df_labeledMovies.loc[given_movie_id, "labels"] # getting the cluster
movies_in_cluster_indices = df_labeledMovies[df_labeledMovies["labels"] == given_movie_cluster].index.tolist()
movies_in_cluster_relevance = df_ContBaseRec.loc[movies_in_cluster_indices] # Filter the relevance data based on movies in the same cluster

# Train the Nearest Neighbors algorithm
nn = NearestNeighbors(n_neighbors=6)  # 5 neighbors + the given movie itself
nn.fit(movies_in_cluster_relevance)
distances, indices = nn.kneighbors([df_ContBaseRec.loc[given_movie_id]]) # finding the nearest neigbors
nearest_neighbor_ids = [movies_in_cluster_indices[i] for i in indices[0]] # Get the IDs of the nearest neighbors
nearest_neighbor_ids.remove(given_movie_id) # Remove the given movie from the list of neighbors

df_labeledMovies.loc[nearest_neighbor_ids]

C:\Users\HamidEsfahlani\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,labels,title,genres
movieId,,,
440,10,Dave (1993),Comedy|Romance
7149,10,Something's Gotta Give (2003),Comedy|Drama|Romance
539,10,Sleepless in Seattle (1993),Comedy|Drama|Romance
93740,10,Salmon Fishing in the Yemen (2011),Comedy|Drama|Romance
140,10,Up Close and Personal (1996),Drama|Romance


# recommending movies within the given cluster with the example "Network (1975)" with the cousin similarity 

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
#specifying the movie
given_movie_id =df_labeledMovies[df_labeledMovies.title.str.contains(r'\bNetwork\b.{0,9}$', case=False)].labels.item()
# Get the relevance vector of the given movie
given_movie_vector = df_ContBaseRec.loc[given_movie_id].values.reshape(1, -1)
# Calculate cosine similarity between the given movie and all other movies
similarity_scores = cosine_similarity(df_ContBaseRec.values, given_movie_vector)
# Get the indices of the top five similar movies & Get the IDs of the top five similar movies
top_similar_movie_ids= df_ContBaseRec.index[similarity_scores.argsort(axis=0)[-6:-1][::-1].squeeze()]
df_labeledMovies.loc[top_similar_movie_ids]

,labels,title,genres
movieId,,,
440,10,Dave (1993),Comedy|Romance
539,10,Sleepless in Seattle (1993),Comedy|Drama|Romance
7149,10,Something's Gotta Give (2003),Comedy|Drama|Romance
93740,10,Salmon Fishing in the Yemen (2011),Comedy|Drama|Romance
140,10,Up Close and Personal (1996),Drama|Romance


# recommending movies based on several movies

## following steps are requiered: 
1. calculating the mean of all the given movies, lets call it "mean_point"
2. find out the nearest neighbors to  "mean_point" 
3. output the 10 nearest neighbors which have the same label (within the same cluster) as the given movies.

In [73]:
# Given movies and clusters
given_movies_ids = [1598, 250, 32, 464, 8765, 568]
given_movies_clusters = df_labeledMovies.loc[given_movies_ids,"labels"].unique()
movies_in_cluster_indices = df_labeledMovies[df_labeledMovies["labels"].isin(given_movies_clusters)].index.tolist()
# Calculate the mean vector of all the given movies
mean_point = df_ContBaseRec.loc[given_movies_ids].mean()
# make a new dataframe consisting only of the movies which are in the clusters of the given movies:
df_new = df_ContBaseRec.loc[df_labeledMovies[df_labeledMovies['labels'].isin(given_movies_clusters)].index]
#concat the mean vector to the dataframe
df_new.loc[(df_new.index[-1])+1] = mean_point
#retrieve index of the last added mean_point
mean_point_index = df_new.index[-1]

In [ ]:
df_labeledMovies.loc[given_movies_ids,"labels"]

movieId
1598     9
250     16
32      11
464      9
8765    11
568      3
Name: labels, dtype: int32

## train a nearest neighbor to this new dataset

In [80]:
nn = NearestNeighbors(n_neighbors=11)  # 5 neighbors + the given movie itself
nn.fit(df_new)
distances, indices = nn.kneighbors([df_new.loc[mean_point_index]]) # finding the nearest neigbors
nearest_neighbor_ids = [df_new.index.tolist()[i] for i in indices[0]] # Get the IDs of the nearest neighbors
nearest_neighbor_ids.remove(mean_point_index) # Remove the given movie from the list of neighbors

df_labeledMovies.loc[nearest_neighbor_ids]

C:\Users\HamidEsfahlani\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,labels,title,genres
movieId,,,
6341,3,"Shape of Things, The (2003)",Drama
320,3,Suture (1993),Film-Noir|Thriller
2741,9,No Mercy (1986),Action|Crime|Thriller
3112,3,'night Mother (1986),Drama
4175,3,Gray's Anatomy (1996),Comedy|Drama
4574,9,Blind Fury (1989),Action|Thriller
479,9,Judgment Night (1993),Action|Crime|Thriller
1910,3,I Went Down (1997),Comedy|Crime|Drama
148,3,"Awfully Big Adventure, An (1995)",Drama


# making a function to recommend movies with Nearest Neighbors

In [285]:
from sklearn.neighbors import NearestNeighbors
#search out the Ids of list of movies 
def search_movieId(given_movies): 
    #given _movies: a list of list of [['movie Title','realese year'] as string 
    given_movies_ids=[]
    for title, year in given_movies:
        given_movie_id = df_movies[df_movies.title.str.contains(r'\b{}\b.*\b{}\b.$'.format(title, year), case=False)].movieId.item()
        given_movies_ids.append(given_movie_id)
    return(given_movies_ids)
    
def NN_ContentBasedRecommendation(given_movies, Number_of_Reccomendations, df, original_num_recs= None):
    if original_num_recs is None: 
        original_num_recs = Number_of_Reccomendations
    given_movies_ids= search_movieId(given_movies) # searching out the movie Id
    mean_point = df.loc[given_movies_ids].mean()
    given_movies_clusters = df_labeledMovies.loc[given_movies_ids,"labels"].unique()
    movies_in_cluster_indices = df_labeledMovies[df_labeledMovies["labels"].isin(given_movies_clusters)].index.tolist()
    df_new = df.loc[df_labeledMovies[df_labeledMovies['labels'].isin(given_movies_clusters)].index]
    df_new.loc[(df_new.index[-1])+1] = mean_point
    mean_point_index = df_new.index[-1]
    
    nn = NearestNeighbors(n_neighbors=Number_of_Reccomendations+2)
    nn.fit(df_new)
    distances, indices = nn.kneighbors([df_new.loc[mean_point_index]]) # finding the nearest neighbors
    nearest_neighbor_ids = [df_new.index.tolist()[i] for i in indices[0]] # Get the IDs of the nearest neighbors
    nearest_neighbor_ids.remove(mean_point_index)  # Remove the mean point from the list of neighbors
    # Remove the given movie from the list of reccomendation
    for i in search_movieId(given_movies):
        if i in nearest_neighbor_ids:
            nearest_neighbor_ids.remove(i)
    if len(nearest_neighbor_ids) < original_num_recs:
        New_Number_of_Reccomendations = original_num_recs+(original_num_recs - len(nearest_neighbor_ids))
        return NN_ContentBasedRecommendation(given_movies, New_Number_of_Reccomendations, df, original_num_recs=Number_of_Reccomendations)
    else:
        return df_labeledMovies.loc[nearest_neighbor_ids]

## making recommendation based on the last five moveis I saw

In [286]:
df_movies = pd.read_csv('movies.csv')

given_movies= [['Akira', '1988'], ['Network', '1976'], ['Prisoners', '2013'], ['Incendies', '2010'], ['red shoes', '1948'], ['metropolis', '1927']]


NN = NN_ContentBasedRecommendation(given_movies, 20,  df_ContBaseRec)

NN


C:\Users\HamidEsfahlani\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,labels,title,genres
movieId,,,
7382,16,I'm Not Scared (Io non ho paura) (2003),Drama|Mystery|Thriller
5177,17,"Magnificent Ambersons, The (1942)",Drama|Romance
25923,14,Great Expectations (1946),Drama
1280,14,Raise the Red Lantern (Da hong deng long gao g...,Drama
923,14,Citizen Kane (1941),Drama|Mystery
26694,14,Ju Dou (1990),Drama
949,14,East of Eden (1955),Drama
3181,16,Titus (1999),Drama
1132,14,Manon of the Spring (Manon des sources) (1986),Drama


# making a function to recommend movies with nearest neigbor training on the whole dataset

In [282]:
#search out the Ids of list of movies 
def search_movieId(given_movies): 
    #given _movies: a list of list of [['movie Title','realese year'] as string 
    given_movies_ids=[]
    for title, year in given_movies:
        given_movie_id = df_movies[df_movies.title.str.contains(r'\b{}\b.*\b{}\b.$'.format(title, year), case=False)].movieId.item()
        given_movies_ids.append(given_movie_id)
    return(given_movies_ids)
    
def NN_ContentBasedRecommendation_Whole(given_movies, Number_of_Reccomendations, df, original_num_recs= None):
    if original_num_recs is None: 
        original_num_recs = Number_of_Reccomendations
    given_movies_ids= search_movieId(given_movies) # searching out the movie Id
    mean_point = df.loc[given_movies_ids].mean()
    given_movies_clusters = df_labeledMovies.loc[given_movies_ids,"labels"].unique()
    movies_in_cluster_indices = df_labeledMovies[df_labeledMovies["labels"].isin(given_movies_clusters)].index.tolist()
    #df_new = df.loc[df_labeledMovies[df_labeledMovies['labels'].isin(given_movies_clusters)].index]
    df.loc[(df.index[-1])+1] = mean_point
    mean_point_index = df.index[-1]
    
    nn = NearestNeighbors(n_neighbors=Number_of_Reccomendations+1)
    nn.fit(df)
    
    distances, indices = nn.kneighbors([df.loc[mean_point_index]]) # finding the nearest neighbors which are present in movies_in_cluster
    nearest_neighbor_ids = []
    for row in indices:
        for i in row:
            if i not in movies_in_cluster_indices:
                continue
            nearest_neighbor_ids.append(df.index[i])
    
    # Remove the given movie from the list of reccomendation
    for i in search_movieId(given_movies):
        if i in nearest_neighbor_ids:
            nearest_neighbor_ids.remove(i)
    if len(nearest_neighbor_ids) < original_num_recs:
        New_Number_of_Reccomendations = Number_of_Reccomendations+(Number_of_Reccomendations - len(nearest_neighbor_ids))
        return NN_ContentBasedRecommendation_Whole(given_movies, New_Number_of_Reccomendations, df, original_num_recs)
    else:
        return df_labeledMovies.loc[nearest_neighbor_ids[:20]]

## making recommendation based on the last five moveis I saw

In [283]:
df_movies = pd.read_csv('movies.csv')

given_movies= [['Akira', '1988'], ['Network', '1976'], ['Prisoners', '2013'], ['Incendies', '2010'], ['red shoes', '1948'], ['metropolis', '1927']]


NN_whole = NN_ContentBasedRecommendation_Whole(given_movies, 20,  df_ContBaseRec)

NN_whole


C:\Users\HamidEsfahlani\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\HamidEsfahlani\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\HamidEsfahlani\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\HamidEsfahlani\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\HamidEsfahlani\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,labels,title,genres
movieId,,,
3736,14,"Ace in the Hole (Big Carnival, The) (1951)",Drama
5062,16,Seconds (1966),Mystery|Sci-Fi|Thriller
4423,4,"Entertainer, The (1960)",Drama
3310,14,"Kid, The (1921)",Comedy|Drama
3342,14,Birdy (1984),Drama|War
1292,14,Being There (1979),Comedy|Drama
7745,17,"Scent of Green Papaya, The (Mùi du du xhan - L...",Drama
3221,17,"Draughtsman's Contract, The (1982)",Drama
6985,17,"Passion of Joan of Arc, The (Passion de Jeanne...",Drama


In [295]:
identical_rows_count = pd.merge(NN, NN_whole, on=CS.columns.tolist(), how='inner').shape[0]
print(f"The proportion of identical rows between the dataframes is: {(identical_rows_count*100)/20} %")

The proportion of identical rows between the dataframes is: 5.0 %


In [288]:
print(f"The model trained on the whole data set has a diversity of {round(NN_whole.labels.nunique()/df_labeledMovies.labels.nunique()*100,2)} %\n"
      f" while the one trained on the smaller data set has a diversity of {round(NN.labels.nunique()/df_labeledMovies.labels.nunique()*100,2)}%")

The model trained on the whole data set has a diversity of 22.22 %
 while the one trained on the smaller data set has a diversity of 16.67%


# making a function to recommend movies with cousin similarity

In [296]:
from sklearn.metrics.pairwise import cosine_similarity
#search out the Ids of list of movies 
def search_movieId(given_movies): 
    #given _movies: a list of list of [['movie Title','realese year'] as string 
    given_movies_ids=[]
    for title, year in given_movies:
        given_movie_id = df_movies[df_movies.title.str.contains(r'\b{}\b.*\b{}\b.$'.format(title, year), case=False)].movieId.item()
        given_movies_ids.append(given_movie_id)
    return(given_movies_ids)
    
def CS_ContentBasedRecommendation(given_movies, Number_of_Reccomendations, df, original_num_recs= None):
    if original_num_recs is None: 
        original_num_recs = Number_of_Reccomendations
    given_movies_ids = search_movieId(given_movies) # searching out the movie Id
    mean_point = df.loc[given_movies_ids].mean()
    given_movies_clusters = df_labeledMovies.loc[given_movies_ids, "labels"].unique()
    movies_in_cluster_indices = df_labeledMovies[df_labeledMovies["labels"].isin(given_movies_clusters)].index.tolist()
    df_new = df.loc[df_labeledMovies[df_labeledMovies['labels'].isin(given_movies_clusters)].index]

    from sklearn.metrics.pairwise import cosine_similarity
    # Get the relevance vector of the given movie
    given_movie_vector = mean_point.values.reshape(1, -1)
    # Calculate cosine similarity between the given movie and all other movies
    similarity_scores = cosine_similarity(df_new.values, given_movie_vector)
    # Get the indices of the top similar movies & Get the IDs of the top similar movies
    top_similar_movie_ids = list(df_new.index[similarity_scores.argsort(axis=0)[-(Number_of_Reccomendations + 1):-1][::-1].squeeze()])
    # Remove the mean point from the list of neighbors
    # top_similar_movie_ids.remove(mean_point_index) 
    # Remove the given movie from the list of recommendations
    for i in search_movieId(given_movies):
        if i in top_similar_movie_ids:
            top_similar_movie_ids.remove(i)
    if len(top_similar_movie_ids) < original_num_recs:
        New_Number_of_Reccomendations = original_num_recs + (original_num_recs - len(top_similar_movie_ids))
        return CS_ContentBasedRecommendation(given_movies, New_Number_of_Reccomendations, df, original_num_recs=Number_of_Reccomendations)
    else:
        return df_labeledMovies.loc[top_similar_movie_ids]

In [297]:
df_movies = pd.read_csv('movies.csv')

given_movies= [['Akira', '1988'], ['Network', '1976'], ['Prisoners', '2013'], ['Incendies', '2010'], ['red shoes', '1948'], ['metropolis', '1927']]


CS = CS_ContentBasedRecommendation(given_movies, 20,  df_ContBaseRec)
CS

,labels,title,genres
movieId,,,
84952,16,Confessions (Kokuhaku) (2010),Drama|Horror
26694,14,Ju Dou (1990),Drama
1280,14,Raise the Red Lantern (Da hong deng long gao g...,Drama
1217,14,Ran (1985),Drama|War
56782,14,There Will Be Blood (2007),Drama|Western
27773,16,Old Boy (2003),Mystery|Thriller
96829,14,"Hunt, The (Jagten) (2012)",Drama
31364,16,Memories of Murder (Salinui chueok) (2003),Crime|Drama|Mystery|Thriller
81591,16,Black Swan (2010),Drama|Thriller


In [298]:
identical_rows_count = pd.merge(CS, NN, on=CS.columns.tolist(), how='inner').shape[0]
print(f"The proportion of identical rows between the dataframes is: {(identical_rows_count*100)/20} %")


The proportion of identical rows between the dataframes is: 40.0 %


# making a function to recommend movies with cousin similarity trained on the whole data set

In [314]:
from sklearn.metrics.pairwise import cosine_similarity
#search out the Ids of list of movies 
def search_movieId(given_movies): 
    #given _movies: a list of list of [['movie Title','realese year'] as string 
    given_movies_ids=[]
    for title, year in given_movies:
        given_movie_id = df_movies[df_movies.title.str.contains(r'\b{}\b.*\b{}\b.$'.format(title, year), case=False)].movieId.item()
        given_movies_ids.append(given_movie_id)
    return(given_movies_ids)
    
def CS_ContentBasedRecommendation_whole(given_movies, Number_of_Reccomendations, df, original_num_recs= None):
    if original_num_recs is None: 
        original_num_recs = Number_of_Reccomendations
    given_movies_ids = search_movieId(given_movies) # searching out the movie Id
    mean_point = df.loc[given_movies_ids].mean()
    given_movies_clusters = df_labeledMovies.loc[given_movies_ids, "labels"].unique()
    movies_in_cluster_indices = df_labeledMovies[df_labeledMovies["labels"].isin(given_movies_clusters)].index.tolist()
    #df_new = df.loc[df_labeledMovies.index]

    # Get the relevance vector of the given movie
    given_movie_vector = mean_point.values.reshape(1, -1)
    # Calculate cosine similarity between the given movie and all other movies
    similarity_scores = cosine_similarity(df.values, given_movie_vector)
    # Get the indices of the top similar movies & Get the IDs of the top similar movies
    top_similar_movie_ids = list(df.index[similarity_scores.argsort(axis=0)[-(Number_of_Reccomendations + 1):-1][::-1].squeeze()]) # Remove the given movie from the list of recommendations
    #top_similar_movie_ids = [i for i in top_similar_movie_ids if i not in search_movieId(given_movies)]
    
    #check if they have the same cluster as the given movies
    top_similar_movie_ids = [i for i in top_similar_movie_ids if i in movies_in_cluster_indices]  
    # Remove the given movie from the list of recommendations
    for i in search_movieId(given_movies):
        if i in top_similar_movie_ids:
            top_similar_movie_ids.remove(i)
    
    if len(top_similar_movie_ids) < original_num_recs:
        New_Number_of_Reccomendations = Number_of_Reccomendations + (original_num_recs - len(top_similar_movie_ids))
        return CS_ContentBasedRecommendation_whole(given_movies, New_Number_of_Reccomendations, df, original_num_recs)
    else:
        return df_labeledMovies.loc[top_similar_movie_ids]

In [315]:
df_movies = pd.read_csv('movies.csv')

given_movies= [['Akira', '1988'], ['Network', '1976'], ['Prisoners', '2013'], ['Incendies', '2010'], ['red shoes', '1948'], ['metropolis', '1927']]


CS_whole = CS_ContentBasedRecommendation_whole(given_movies, 20,  df_ContBaseRec)
CS_whole

,labels,title,genres
movieId,,,
94466,16,Black Mirror (2011),Drama|Sci-Fi
84952,16,Confessions (Kokuhaku) (2010),Drama|Horror
26694,14,Ju Dou (1990),Drama
1280,14,Raise the Red Lantern (Da hong deng long gao g...,Drama
1217,14,Ran (1985),Drama|War
56782,14,There Will Be Blood (2007),Drama|Western
27773,16,Old Boy (2003),Mystery|Thriller
96829,14,"Hunt, The (Jagten) (2012)",Drama
31364,16,Memories of Murder (Salinui chueok) (2003),Crime|Drama|Mystery|Thriller


In [316]:
identical_rows_count = pd.merge(CS, CS_whole, on=CS.columns.tolist(), how='inner').shape[0]
print(f"The proportion of identical rows between the dataframes is: {(identical_rows_count*100)/20} %")


The proportion of identical rows between the dataframes is: 95.0 %


In [317]:
print(f"The model trained on the whole data set has a diversity of {round(CS_whole.labels.nunique()/df_labeledMovies.labels.nunique()*100,2)} %\n"
      f" while the one trained on the smaller data set has a diversity of {round(CS.labels.nunique()/df_labeledMovies.labels.nunique()*100,2)}%")

The model trained on the whole data set has a diversity of 11.11 %
 while the one trained on the smaller data set has a diversity of 11.11%


Conclusion:

it has an efect if nneghbors is trained on the whole set or not, but not much cousin similarity. Anyway we will traine our algorithm on the whole dataset